Importação de Bibliotecas

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import numpy as np
import matplotlib.pyplot as plt

2023-11-04 00:52:16.591766: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-04 00:52:16.591797: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-04 00:52:16.591814: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-04 00:52:16.596856: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
!pip install scipy


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


Carrregamentos de dados

In [3]:
train_dir = 'train/'
test_dir = 'test/'


In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    device = "GPU:0"
else:
    device = "CPU:0"

2023-11-04 00:52:25.516340: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-04 00:52:25.520379: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-04 00:52:25.520509: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Normalização de dados

In [5]:
data_augmentation = keras.Sequential([
    layers.Resizing(64, 64),
    layers.RandomFlip("horizontal"),  # Horizontal flip data augmentation
    layers.Rescaling(1.0 / 255)  # Normalize pixel values
])


2023-11-04 00:52:27.864320: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-04 00:52:27.864484: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-04 00:52:27.864581: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [6]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=data_augmentation,
    validation_split=0.2  # 80% training, 20% validation
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=64,
    class_mode='categorical',
    subset='training'  # Specify the subset for training
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=64,
    class_mode='categorical',
    subset='validation'  # Specify the subset for validation
)

Found 4965 images belonging to 4 classes.
Found 1240 images belonging to 4 classes.


In [7]:
test_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=data_augmentation
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),
    batch_size=64,
    class_mode='categorical'
)

Found 1608 images belonging to 4 classes.


Modelo da convulução

In [8]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(4, activation='softmax')
])


Treinamento do modelo

Teste do modelo

In [9]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])# Train the model
num_epochs = 20
model.fit(train_generator, epochs=num_epochs)

# Save the trained model


Epoch 1/20


2023-11-04 00:52:40.549329: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2023-11-04 00:52:41.177851: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1e3c3d50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-04 00:52:41.177869: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-11-04 00:52:41.183616: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-04 00:52:41.261925: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


78/78 [==============================] - 26s 293ms/step - loss: 0.9765 - accuracy: 0.5122
Epoch 2/20
78/78 [==============================] - 24s 308ms/step - loss: 0.6328 - accuracy: 0.6969
Epoch 3/20
78/78 [==============================] - 24s 306ms/step - loss: 0.5410 - accuracy: 0.7511
Epoch 4/20
78/78 [==============================] - 25s 314ms/step - loss: 0.4800 - accuracy: 0.7742
Epoch 5/20
78/78 [==============================] - 24s 301ms/step - loss: 0.4227 - accuracy: 0.8044
Epoch 6/20
78/78 [==============================] - 25s 316ms/step - loss: 0.3815 - accuracy: 0.8356
Epoch 7/20
78/78 [==============================] - 24s 301ms/step - loss: 0.3699 - accuracy: 0.8348
Epoch 8/20
78/78 [==============================] - 25s 315ms/step - loss: 0.3223 - accuracy: 0.8616
Epoch 9/20
78/78 [==============================] - 23s 291ms/step - loss: 0.2675 - accuracy: 0.8906
Epoch 10/20
78/78 [==============================] - 22s 288ms/step - loss: 0.2190 - accuracy: 0.9112


KeyboardInterrupt: 

In [10]:
model.save('trained_model_tf6.h5')

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Conversao do modelo

In [13]:
class_labels = ['cadeira', 'close', 'mesa','open']

image_path = 'test/mesa/31DHLGBWBK5D.jpg'  # Change the path to your test image
image = keras.preprocessing.image.load_img(image_path, target_size=(64, 64))
input_data = keras.preprocessing.image.img_to_array(image)
input_data = input_data / 255.0  # Normalize pixel values

input_data = input_data[np.newaxis, ...]  # Add a batch dimension
predicted_probabilities = model.predict(input_data)

predicted_class = class_labels[np.argmax(predicted_probabilities)]
print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 12ms/step
Predicted class: mesa
